In [106]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
import spacy

from gensim.utils import simple_preprocess

from gensim.models import Phrases
from gensim.models.phrases import Phraser

from gsdmm import MovieGroupProcess

ImportError: cannot import name 'triu' from 'scipy.linalg' (/home/rodrigochavoni/.local/lib/python3.10/site-packages/scipy/linalg/__init__.py)

In [93]:
df = pd.read_csv('data/tweets_prefeito_vacina.csv')

In [94]:
df['text']

0        O Brasil vai decidir neste ano se é tarado por...
1        A propósito, doutor Augusto Aras: @MPF_PGR Vos...
2        De novo, Bolsonaro espalha mentiras sobre as v...
3        Gente. É óbvio que Bolsonaro tá vacinado. Óbvi...
4        2) Johnson perguntou se Bolsonaro tinha tomado...
                               ...                        
17296    Folgamos em saber que Dona Olinda não virou ja...
17297    E, como jacaré parado vira bolsa, bora lutar! ...
17298    Acompanhei ao longo da manhã a CPI da Covid co...
17299    Não seremos “cobaias” de um desgoverno. Não é ...
17300    Minha mãe está feliz da vida pq ontem foi o “g...
Name: text, Length: 17301, dtype: object

In [101]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
tokens_text = list(sent_to_words(df['text']))

NameError: name 'gensim' is not defined

In [ ]:
def make_n_grams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)  # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    bigrams_text = [bigram_mod[doc] for doc in texts]
    trigrams_text =  [trigram_mod[bigram_mod[doc]] for doc in bigrams_text]
    return trigrams_text

In [100]:
tokens_reviews = make_n_grams(tokens_text)

SyntaxError: closing parenthesis '}' does not match opening parenthesis '(' (3070624635.py, line 1)

In [95]:
# Normalização dos dados (remoção de stopwords, pontuação, etc)
def normalize_text(text: str):
    stopword_list = list(nltk.corpus.stopwords.words('portuguese')) + ['https', 'anos', 'hoje', 'brasil', 'dia', 'todos', 'mil', 'vamos', 'ainda', 'sobre', 'pra', 'milhões', 'vai']
    return  " ".join([word for word in word_tokenize(text.lower()) if word not in stopword_list and word.isalpha()])

# Geração de coluna normalizada com base em outra coluna
def generate_normalized_column(data: pd.DataFrame, from_column: str):
    new_column_name = f"{from_column}_Normalized"
    print(f"New column name: {new_column_name}")
    data[new_column_name] = data.apply(lambda linha: normalize_text(str(linha[from_column])), axis = 1)
    return new_column_name


normalized_summary_column_name = generate_normalized_column(data=df, from_column="text")

New column name: text_Normalized


In [96]:
df['text_Normalized']

0                    decidir neste ano tarado vacina morte
1        propósito doutor augusto aras vossa excelência...
2        novo bolsonaro espalha mentiras vacinas desta ...
3        gente óbvio bolsonaro tá vacinado óbvio esfreg...
4        johnson perguntou bolsonaro tomado vacina nego...
                               ...                        
17296              folgamos saber dona olinda virou jacaré
17297    jacaré parado vira bolsa bora lutar impeachmen...
17298    acompanhei longo manhã cpi covid participação ...
17299    cobaias desgoverno justo confusão aflição inse...
17300    mãe feliz vida pq ontem grande vacinou virou j...
Name: text_Normalized, Length: 17301, dtype: object

In [52]:
# Adição de coluna de polaridade e subjetividade com base em outra coluna
def generate_polarity_and_subjectivity_columns(data: pd.DataFrame, from_column: str):
    new_polarity_column_name = f"{from_column}_Polarity"
    data[new_polarity_column_name] = data[from_column].apply(lambda text: TextBlob(text).sentiment.polarity)

    new_subjectivity_column_name = f"{from_column}_Subjectivity"
    data[new_subjectivity_column_name] = data[from_column].apply(lambda text: TextBlob(text).sentiment.subjectivity)

    return new_polarity_column_name, new_subjectivity_column_name

normalized_summary_polarity_column_name, normalized_summary_subjectivity_column_name = generate_polarity_and_subjectivity_columns(data=df, from_column=normalized_summary_column_name)

In [55]:
# Aplicação de lematização em coluna, retornando uma lista de palavras em sua forma raiz
def apply_lemmatization_to_column(data: pd.DataFrame, column: str, allowed_postags: list[str] = ['NOUN', 'ADJ', 'VERB', 'ADV']):
    nlp = spacy.load("en_core_web_sm")

    lemma_column_words = []
    for words in data[column]:
        doc = nlp(" ".join(words))
        lemma_column_words.append(" ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags]))

    return lemma_column_words
#if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags
df['lemma_column'] = apply_lemmatization_to_column(data=df, column=normalized_summary_column_name)
df['text_lemma'] = apply_lemmatization_to_column(data=df, column='text')

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17301 entries, 0 to 17300
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             17301 non-null  int64  
 1   created_at                             17301 non-null  object 
 2   text                                   17301 non-null  object 
 3   public_metrics.like_count              17301 non-null  int64  
 4   public_metrics.quote_count             17301 non-null  int64  
 5   public_metrics.reply_count             17301 non-null  int64  
 6   public_metrics.retweet_count           17301 non-null  int64  
 7   author.id                              17301 non-null  int64  
 8   author.created_at                      17301 non-null  object 
 9   author.description                     17289 non-null  object 
 10  author.location                        16649 non-null  object 
 11  au

In [57]:
df.to_csv('data/processData.csv', index=False)

In [97]:
mgp = MovieGroupProcess(K=20, alpha=0.01, beta=0.01, n_iters=30)
vocab = set(x for x in normalized_summary_column_name)
vocab_size = len(vocab)
output = mgp.fit(normalized_summary_column_name, vocab_size)


In stage 0: transferred 10 clusters with 6 clusters populated
In stage 1: transferred 12 clusters with 7 clusters populated
In stage 2: transferred 9 clusters with 8 clusters populated
In stage 3: transferred 11 clusters with 7 clusters populated
In stage 4: transferred 7 clusters with 6 clusters populated
In stage 5: transferred 10 clusters with 7 clusters populated
In stage 6: transferred 9 clusters with 5 clusters populated
In stage 7: transferred 10 clusters with 5 clusters populated
In stage 8: transferred 10 clusters with 6 clusters populated
In stage 9: transferred 11 clusters with 6 clusters populated
In stage 10: transferred 10 clusters with 6 clusters populated
In stage 11: transferred 11 clusters with 7 clusters populated
In stage 12: transferred 10 clusters with 7 clusters populated
In stage 13: transferred 11 clusters with 6 clusters populated
In stage 14: transferred 11 clusters with 6 clusters populated
In stage 15: transferred 12 clusters with 8 clusters populated
In st

In [98]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster,sort_dicts))

In [99]:

doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('\nMost important clusters (by number of docs inside):', top_index)
# show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 10)

Number of documents per topic : [0 1 1 4 3 0 2 0 0 0 0 0 0 2 0 0 0 2 0 0]

Most important clusters (by number of docs inside): [ 3  4 17 13  6  1  2 19  8  5]

Cluster 3 : [('o', 1), ('l', 1), ('z', 1), ('d', 1)]

Cluster 4 : [('e', 2), ('_', 1)]

Cluster 17 : [('t', 2)]

Cluster 13 : [('r', 1), ('a', 1)]

Cluster 6 : [('x', 1), ('i', 1)]

Cluster 1 : [('N', 1)]

Cluster 2 : [('m', 1)]

Cluster 19 : []

Cluster 8 : []

Cluster 5 : []
